<a href="https://colab.research.google.com/github/jdmartinezrs/incomesAndBillsProject/blob/main/Copia_de_Trabajo1ParaEntregar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**1. Importar Dataset**

In [2]:
from google.colab import files
from google.colab import drive
from sklearn.neighbors import KNeighborsClassifier
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
sheet_id ='1hSCgq0sKpE_O2XI55UfuVPkA6lYoySZSCE7KWhdxWNY'
sheet_url = f'https://docs.google.com/spreadsheets/d/{sheet_id}/export?format=csv'

dp = pd.read_csv(sheet_url, header=5)
print(dp.head())

           Fecha  ITEM CODIGO DE EQUIPO Tipo de Vehiculo fecha de revision  \
0            NaN   1.0  VE-DT-CH-XMD284     CARRO TANQUE         5/23/2025   
1            NaN   2.0  VE-DT-FL-XMD588      DOBLETROQUE         5/23/2025   
2            NaN   3.0  VE-DT-FL-SUE755      DOBLETROQUE         5/23/2025   
3  5/23/25 17:32   4.0  VE-DT-FL-WFD547      DOBLETROQUE         5/23/2025   
4  5/23/25 17:32   5.0  VE-DT-IN-TAV909      DOBLETROQUE         5/23/2025   

  Km/ Hr Plan  Fecha Ult Pre-Operacional  Dias Pend   Ult Km/Hr Km/Hr Cambio  \
0     10,000                     45797.0        3.0    320,832      316,580    
1     10,000                     45190.0      610.0    209,543      199,850    
2      8,000                     45797.0        3.0    431,842       429,813   
3     10,000                     45797.0        3.0    269,735       263,671   
4     10,000                     45797.0        3.0    288,404      287,932    

   Fecha Cambio  Fecha Prox Cambio  Unnamed: 12 Km

In [4]:
data = {
    'km_hr_pend': pd.to_numeric(dp['Km/Hr Pend'], errors='coerce'),
    'progreso': pd.to_numeric(dp['Progreso2'], errors='coerce')
}
dx = pd.DataFrame(data)

In [24]:
print("Contenido original de dp:")
print(dp.head())

print("\nContenido procesado de dx:")
print(dx.head())
print("\nCantidad de filas en dx antes de limpieza:", len(dx))

# Aplicar limpieza
dx = dx.dropna()


dp = dp.replace(["N/A", "--"], np.nan)


print("\nCantidad de filas en dx después de limpieza:", len(dx))



Contenido original de dp:
           Fecha  ITEM CODIGO DE EQUIPO Tipo de Vehiculo fecha de revision  \
0            NaN   1.0  VE-DT-CH-XMD284     CARRO TANQUE         5/23/2025   
1            NaN   2.0  VE-DT-FL-XMD588      DOBLETROQUE         5/23/2025   
2            NaN   3.0  VE-DT-FL-SUE755      DOBLETROQUE         5/23/2025   
3  5/23/25 17:32   4.0  VE-DT-FL-WFD547      DOBLETROQUE         5/23/2025   
4  5/23/25 17:32   5.0  VE-DT-IN-TAV909      DOBLETROQUE         5/23/2025   

  Km/ Hr Plan  Fecha Ult Pre-Operacional  Dias Pend   Ult Km/Hr Km/Hr Cambio  \
0     10,000                     45797.0        3.0    320,832      316,580    
1     10,000                     45190.0      610.0    209,543      199,850    
2      8,000                     45797.0        3.0    431,842       429,813   
3     10,000                     45797.0        3.0    269,735       263,671   
4     10,000                     45797.0        3.0    288,404      287,932    

   Fecha Cambio  Fecha P

In [29]:
conditions = [
    (dx['progreso'] > 90) | (dx['km_hr_pend'] < 0),
    (dx['progreso'] > 70) & (dx['progreso'] <= 90),
    (dx['progreso'] <= 70)
]
choices = ['Alto', 'Medio', 'Bajo']


In [34]:
dx['riesgo'] = np.select(conditions, choices, default='Desconocido')



In [35]:
x = dx[['km_hr_pend', 'progreso']]
y = dx['riesgo']

In [38]:
print("Cantidad total de muestras:", len(x))

# Split solo si hay suficientes datos y más de una clase
if len(x) > 0 and len(y.unique()) > 1:
    x_train, x_test, y_train, y_test = train_test_split(
        x, y, test_size=0.2, random_state=42, stratify=y
    )
    print("Split exitoso.")
else:
    print("No hay suficientes datos para hacer el split. Revisa tus filtros.")

Cantidad total de muestras: 0
No hay suficientes datos para hacer el split. Revisa tus filtros.


In [37]:
x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.2, random_state=42, stratify=y
)

ValueError: With n_samples=0, test_size=0.2 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

In [ ]:
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [ ]:
modelo_knn = KNeighborsClassifier(n_neighbors=5)
modelo_knn.fit(x_train, y_train)

In [ ]:
y_pred = modelo_knn.predict(x_test)
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

1.1 Linpieza de datos

In [ ]:
for col in dp.columns:
    if dp[col].dtype == 'object':
        dp[col] = dp[col].fillna('N/A')
    else:
        dp[col] = dp[col].fillna(0)

# Mostrar resultado
print(dp.head())

def eliminar_filas_na(dp):
    """
    Elimina filas que contienen 'N/A' en cualquier columna.
    """
    dp = dp[~dp.apply(lambda row: 'N/A' in row.values, axis=1)]
    return dp
dp = eliminar_filas_na(dp)

def convertir_coma_a_punto(dp, columna):
    """
    Convierte los valores de una columna reemplazando las comas por puntos
    y asegurando que sean numéricos.
    """
    dp[columna] = dp[columna].astype(str).str.replace(',', '.')
    dp[columna] = pd.to_numeric(dp[columna], errors='coerce')
    return dp

dp = convertir_coma_a_punto(dp, 'Km/Hr Pend')





           Fecha  ITEM CODIGO DE EQUIPO Tipo de Vehiculo fecha de revision  \
3  5/23/25 17:32   4.0  VE-DT-FL-WFD547      DOBLETROQUE         5/23/2025   
4  5/23/25 17:32   5.0  VE-DT-IN-TAV909      DOBLETROQUE         5/23/2025   
5  5/23/25 17:32   6.0  VE-DT-IN-TAW674      DOBLETROQUE         5/23/2025   
6  5/23/25 17:32   7.0  VE-DT-IN-TAW675      DOBLETROQUE         5/23/2025   
7  5/23/25 17:32   8.0  VE-DT-IN-TAW676      DOBLETROQUE         5/23/2025   

  Km/ Hr Plan  Fecha Ult Pre-Operacional  Dias Pend   Ult Km/Hr Km/Hr Cambio  \
3     10,000                     45797.0        3.0    269,735       263,671   
4     10,000                     45797.0        3.0    288,404      287,932    
5     12,000                     45427.0      373.0     86,614       76,403    
6     12,000                     45498.0      302.0    357,846      357,292    
7     12,000                     45402.0      398.0    375,905      376,257    

   Fecha Cambio  Fecha Prox Cambio  Unnamed: 12 Km

**1.1 Importar las liberias necesarias**

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
import pandas as pd

**2. Crear un conjunto de datos**

In [ ]:
data = {
    'km_hr_pend': dp['Km/Hr Pend'].tolist(),
    #'dias_pend': dp['Dias Pend'].tolist(),
    'progreso2': dp['Progreso2'].tolist()
}
dx = pd.DataFrame(data)
print(dx)

    km_hr_pend   progreso2
0        3.936   39.360000
1        9.528   95.280000
2        1.789    0.000000
3       11.446    0.000000
4       12.352    0.000000
5        7.034   70.340000
6        5.675    0.000000
7        4.015   40.150000
8       12.049    0.000000
9       12.006    0.000000
10       4.605   46.050000
11       9.655    0.000000
12       7.087    0.000000
13       5.463   54.630000
14       3.481   34.810000
15       9.160   91.600000
16       2.769   55.380000
17       5.000  100.000000
18       3.499   58.316667
19       2.410   40.166667
20       6.274    0.000000
21       6.665    0.000000
22       7.545   75.450000
23      12.000  100.000000


In [ ]:
def clasificar_riesgo(progreso2, km_hr_pend):
    if progreso2 > 90 or km_hr_pend < 0:
        return 'Alto'
    elif 70 <= progreso2 <= 90:
        return 'Medio'
    else:
        return 'Bajo'

dx['riesgo'] = dx.apply(lambda row: clasificar_riesgo(row['progreso2'], row['km_hr_pend']), axis=1)


In [ ]:

import numpy as np

dx['km_hr_pend'] = pd.to_numeric(dx['km_hr_pend'], errors='coerce')
#dx['dias_pend'] = pd.to_numeric(dx['dias_pend'], errors='coerce')
dx['progreso'] = pd.to_numeric(dx['progreso2'], errors='coerce')

# Definir condiciones de riesgo
conditions = [
    (dx['progreso'] > 90) | (dx['km_hr_pend'] < 0),
    (dx['progreso'] > 70) & (dx['progreso'] <= 90),
    (dx['progreso'] <= 70)
]

choices = ['Alto', 'Medio', 'Bajo']

# Especificar un valor por defecto que también sea string
dx['riesgo'] = np.select(conditions, choices, default='Desconocido')

# Mostrar resultado
print(dx[['km_hr_pend',  'progreso', 'riesgo']])
#print(dx[['km_hr_pend', 'dias_pend', 'progreso', 'riesgo']])


    km_hr_pend    progreso riesgo
0        3.936   39.360000   Bajo
1        9.528   95.280000   Alto
2        1.789    0.000000   Bajo
3       11.446    0.000000   Bajo
4       12.352    0.000000   Bajo
5        7.034   70.340000  Medio
6        5.675    0.000000   Bajo
7        4.015   40.150000   Bajo
8       12.049    0.000000   Bajo
9       12.006    0.000000   Bajo
10       4.605   46.050000   Bajo
11       9.655    0.000000   Bajo
12       7.087    0.000000   Bajo
13       5.463   54.630000   Bajo
14       3.481   34.810000   Bajo
15       9.160   91.600000   Alto
16       2.769   55.380000   Bajo
17       5.000  100.000000   Alto
18       3.499   58.316667   Bajo
19       2.410   40.166667   Bajo
20       6.274    0.000000   Bajo
21       6.665    0.000000   Bajo
22       7.545   75.450000  Medio
23      12.000  100.000000   Alto


In [ ]:
print(dx.columns)


Index(['km_hr_pend', 'progreso2', 'riesgo', 'progreso'], dtype='object')


3.Crear y preparar los datos

In [ ]:
x = dx[['km_hr_pend', 'progreso2']]
y = dx['riesgo']


4. Dividir los datos

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42, stratify=y)


5. Crear y entrenar el modelo

In [ ]:
modelo_knn = KNeighborsClassifier(n_neighbors=3)
modelo_knn.fit(x_train, y_train)


KNeighborsClassifier(n_neighbors=3)

In [ ]:
y_pred = modelo_knn.predict(x_test)

In [ ]:
modelo_knn = KNeighborsClassifier(n_neighbors=5)
modelo_knn.fit(x_train, y_train)
y_pred = modelo_knn.predict(x_test)


In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [ ]:
print(y_pred)  # Verifica si el modelo está prediciendo solo 'Bajo'
print(y_train.value_counts())

['Bajo' 'Bajo' 'Bajo' 'Bajo' 'Alto']
riesgo
Bajo     14
Alto      3
Medio     2
Name: count, dtype: int64


In [ ]:
print(y_test.value_counts())  # Ver cantidad de cada clase en los datos de prueba


riesgo
Bajo    4
Alto    1
Name: count, dtype: int64


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

        Alto       1.00      1.00      1.00         1
        Bajo       1.00      1.00      1.00         4

    accuracy                           1.00         5
   macro avg       1.00      1.00      1.00         5
weighted avg       1.00      1.00      1.00         5

[[1 0]
 [0 4]]


In [ ]:
import numpy as np

In [ ]:

import numpy as np

dx['km_hr_pend'] = pd.to_numeric(dx['km_hr_pend'], errors='coerce')
#dx['dias_pend'] = pd.to_numeric(dx['dias_pend'], errors='coerce')
dx['progreso'] = pd.to_numeric(dx['progreso2'], errors='coerce')

# Definir condiciones de riesgo
conditions = [
    (dx['progreso'] > 90) | (dx['km_hr_pend'] < 0),
    (dx['progreso'] > 70) & (dx['progreso'] <= 90),
    (dx['progreso'] <= 70)
]

choices = ['Alto', 'Medio', 'Bajo']

# Especificar un valor por defecto que también sea string
dx['riesgo'] = np.select(conditions, choices, default='Desconocido')

# Mostrar resultado
print(dx[['km_hr_pend',  'progreso', 'riesgo']])
#print(dx[['km_hr_pend', 'dias_pend', 'progreso', 'riesgo']])


    km_hr_pend    progreso riesgo
0        3.936   39.360000   Bajo
1        9.528   95.280000   Alto
2        1.789    0.000000   Bajo
3       11.446    0.000000   Bajo
4       12.352    0.000000   Bajo
5        7.034   70.340000  Medio
6        5.675    0.000000   Bajo
7        4.015   40.150000   Bajo
8       12.049    0.000000   Bajo
9       12.006    0.000000   Bajo
10       4.605   46.050000   Bajo
11       9.655    0.000000   Bajo
12       7.087    0.000000   Bajo
13       5.463   54.630000   Bajo
14       3.481   34.810000   Bajo
15       9.160   91.600000   Alto
16       2.769   55.380000   Bajo
17       5.000  100.000000   Alto
18       3.499   58.316667   Bajo
19       2.410   40.166667   Bajo
20       6.274    0.000000   Bajo
21       6.665    0.000000   Bajo
22       7.545   75.450000  Medio
23      12.000  100.000000   Alto


**3.Crear y preparar los datos**


In [ ]:
#PENDIENTE POR HACER: MOSTRAR EL TIPO DE VEHICULO Y EL RIESGO